In [1]:
import requests
import numpy as np
import pandas as pd
import json
import csv

In [2]:
API_KEY = 'your_api_key'
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

Define a function to search wheelchair accessible businesses in London

In [3]:
def search(category, limit, offset):
    url_params = {
        'location': 'London',
        'categories': category,
        'attributes': 'wheelchair_accessible',
        'limit': limit,
        'offset': offset
    } 
    response = requests.get(url = ENDPOINT, params = url_params, headers = HEADERS)
    business_data = response.json()
    return business_data

Define a function to search businesses in London

In [4]:
def nonsearch(category, limit):
    url_params = {
        'location': 'London',
        'categories': category,
        'limit': limit,
    } 
    response = requests.get(url = ENDPOINT, params = url_params, headers = HEADERS)
    business_data = response.json()
    return business_data

In [21]:
number = search('eventservices', 1, 0)

In [22]:
number['total']

53

Define a function to retrieve up to 1000 businesses by setting results and offset, and clean the data returned into DataFrame

In [7]:
def CatFrame(category):
    #
    check = search(category, 50, 0)
    total = check['total']
    dict_out = []
    results = 50
    offset = 0
    full_requests = int(np.floor(total/results))
    remaining_requests = total%results
    for i in range(1, full_requests + 2):
        if i <= full_requests:
            request = search(category, results, offset)
        if i > full_requests:
            request = search(category, remaining_requests, offset)
        offset += results
        dict_out.extend(request['businesses'])
    df = pd.DataFrame.from_dict(dict_out)
    df1 = pd.concat([df.drop(['coordinates'], axis=1), df['coordinates'].apply(pd.Series)], axis=1)
    df2 = pd.concat([df1.drop(['location'], axis=1), df1['location'].apply(pd.Series)], axis=1)
    df3 = pd.concat([df2.drop(['categories'], axis=1), df2['categories'].apply(pd.Series)], axis=1)
    df4 = df3.rename(columns={0: 'category0', 1: 'category1', 2: 'category2'}, inplace=True)
    final = pd.concat([df4.drop(['category0'], axis=1), final['category0'].apply(pd.Series)], axis=1)
    return final

In [10]:
categories = ['active', 'arts', 'auto', 'beautysvc', 'bicycles', 'education', 'eventservices', 'financialservices', 'food', 'health', 'homeservices', 'hotelstravel', 'localflavor', 'localservices', 'massmedia', 'nightlife', 'pets', 'professional', 'publicservicesgovt', 'realestate', 'religiousorgs', 'restaurants', 'shopping']
#Categories: 
#1.Sports & Leisure (active, All); 
#2.Arts & Entertainment (arts, All); 
#3.Car & Motorcycle (auto, All); 
#4.Beauty & Spas (beautysvc, All); 
#5.Bicycles (bicycles, [CZ, PT, PL, DK]); 
#6.Education (education, All); 
#7.Event Planning & Event Services (eventservices, All); 
#8.Financial Services (financialservices, All); 
#9.Food (food, All); 
#10.Health & Medical (health, All); 
#11.Home Services (homeservices, All); 
#12.Hotel & Travel (hotelstravel, All); 
#13.Local Flavour (localflavor, All); 
#14.Local Services (localservices, All); 
#15.Mass Media (massmedia, All); 
#16.Nightlife (nightlife, All); 
#17.Pets (pets, All); 
#18.Professional Services (professional, All); 
#19.Government & Public Services (publicservicesgovt, All); 
#20.Property (realestate, All); 
#21.Religious Organisations (religiousorgs, All); 
#22.Restaurants (restaurants, All); 
#23.Shopping (shopping, All)

In [ ]:
final = CatFrame('publicservicesgovt')

In [ ]:
final

Transfer the DataFrame into csv 

In [16]:
final.to_csv('publicservicesgovt.csv')